---
toc: true
layout: post
title: Daily Life
description: Users experience a full day of living with diabetes by making lifestyle choices.
categories: [Game Documentation]
---


# Glucose Hero : A Day-in-the-Life Simulation Game

An interactive simulation game where players experience a full day living with diabetes by making lifestyle choices (e.g., meals, activities, sleep). A virtual Dexcom CGM meter shows glucose level changes based on the player's decisions. Pop-up alerts and tips mimic real Dexcom device behavior to reinforce learning and drive awareness of continuous glucose monitoring (CGM).

---

##  Project Goals & Objectives

- **Educate** players on how daily behaviors affect glucose levels  
- **Increase awareness** of Dexcom’s real-time glucose tracking advantages  
- **Boost engagement** through gamified diabetes management  
- **Encourage** healthy decisions through trial, error, and reinforcement  

---

##  Game Design & Mechanics

###  Simulated Day Structure

- **Phases**: Morning → Afternoon → Evening → Night  
- Each phase includes **choice cards** (e.g., "Eat pancakes or eggs?", "Take a walk or nap?")  
- Choices adjust glucose levels via simplified models

###  Glucose Logic (Simplified)

```js
// Example impact logic (pseudo-code)
if (choice === "pancakes") glucose += 30;
if (choice === "walk") glucose -= 15;
```

- Visual meter updates dynamically
- Dexcom-style alerts and arrows simulate CGM feedback

---

##  UI/UX Components

- **TimeOfDaySelector** — switches day phases  
- **ChoiceCard** — renders lifestyle options  
- **CGMGraph** — animated glucose meter with trend arrows  
- **FeedbackTips** — Dexcom-inspired popups and facts  
- **Storage** — game state stored in `localStorage` or React `useState`

---

##  Backend API (Flask)

### Key Endpoints

| Endpoint             | Method | Description                         |
|----------------------|--------|-------------------------------------|
| `/api/glucose-log`   | POST   | Log a user action + glucose impact |
| `/api/glucose-log`   | GET    | Retrieve all glucose logs          |
| `/api/checklist`     | POST   | Add user checklist item            |
| `/api/checklist`     | GET    | Retrieve checklist by user         |
| `/api/checklist`     | PUT    | Update checklist item              |
| `/api/checklist`     | DELETE | Delete checklist item              |

---

## 🧾 GlucoseLog Model (SQLAlchemy)

```python
class GlucoseLog(db.Model):
    user_id = db.Column(db.Integer, nullable=False)
    action = db.Column(db.String(255), nullable=False)
    impact = db.Column(db.Integer, nullable=False)
    glucose_level = db.Column(db.Integer, nullable=False)
```

**Example Entries**:
- 🥞 Eat pancakes with syrup → +30 → glucose = 130  
- 🚶‍♂️ Take a walk → -15 → glucose = 115  
- 🍗 Grilled chicken & veggies → +5 → glucose = 105  

---

## Full API

```python
from flask import Blueprint, request, jsonify, g
from flask_restful import Api, Resource
from __init__ import db
from model.GlucoseLog import GlucoseLog
from api.jwt_authorize import token_required

# Blueprint for Checklist API
checklist_api = Blueprint('checklist_api', __name__, url_prefix='/api')
api = Api(checklist_api)

class ChecklistAPI:
    class _Checklist(Resource):
        @token_required()
        def post(self):
            data = request.get_json()
            current_user = g.current_user

            if not data or 'item_name' not in data:
                return {'message': 'Missing item_name'}, 400

            item = GlucoseLog(
                user_id=current_user.id,
                item_name=data['item_name'],
                is_checked=data.get('is_checked', False)
            )
            item.create()
            return jsonify(item.read())

        @token_required()
        def get(self):
            user_id = request.args.get('user_id')
            if not user_id:
                return {'message': 'Missing user_id'}, 400

            items = GlucoseLog.query.filter_by(user_id=user_id).all()
            return jsonify([item.read() for item in items])

        @token_required()
        def put(self):
            data = request.get_json()
            if not data or 'id' not in data or 'is_checked' not in data:
                return {'message': 'Missing fields'}, 400

            item = GlucoseLog.query.get(data['id'])
            if not item:
                return {'message': 'Item not found'}, 404

            item.is_checked = data['is_checked']
            db.session.commit()
            return {'message': 'Item updated successfully'}, 200

        @token_required()
        def delete(self):
            data = request.get_json()
            if not data or 'id' not in data:
                return {'message': 'Missing id'}, 400

            item = ChecklistItem.query.get(data['id'])
            if not item:
                return {'message': 'Item not found'}, 404

            db.session.delete(item)
            db.session.commit()
            return {'message': 'Item deleted successfully'}, 200

    class _GlucoseLog(Resource):
        @token_required()
        def post(self):
            data = request.get_json()
            current_user = g.current_user

            if not data or 'action' not in data or 'impact' not in data or 'glucose_level' not in data:
                return {'message': 'Missing required fields'}, 400

            log = GlucoseLog(
                user_id=current_user.id,
                action=data['action'],
                impact=data['impact'],
                glucose_level=data['glucose_level']
            )
            log.create()
            return jsonify(log.read())

        @token_required()
        def get(self):
            current_user = g.current_user
            logs = GlucoseLog.query.filter_by(user_id=current_user.id).order_by(GlucoseLog.id.desc()).all()
            return jsonify([log.read() for log in logs])

    api.add_resource(_Checklist, '/checklist')
    api.add_resource(_GlucoseLog, '/glucose-log')
```
---

## Full Model
```python
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from sqlalchemy.orm import relationship
from __init__ import db

class GlucoseLog(db.Model):
    __tablename__ = 'glucose_logs'

    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, nullable=False)
    action = db.Column(db.String(255), nullable=False)
    impact = db.Column(db.Integer, nullable=False)
    glucose_level = db.Column(db.Integer, nullable=False)

    def __init__(self, user_id, action, impact, glucose_level):
        self.user_id = user_id
        self.action = action
        self.impact = impact
        self.glucose_level = glucose_level

    def create(self):
        db.session.add(self)
        db.session.commit()

    def read(self):
        return {
            "id": int(self.id),
            "user_id": int(self.user_id),
            "action": self.action,
            "impact": int(self.impact),
            "glucose_level": int(self.glucose_level)
        }

    def update(self):
        db.session.add(self)
        db.session.commit()

    def delete(self):
        db.session.delete(self)
        db.session.commit()

def initGlucoseLogs():
    sample_logs = [
        {"user_id": 1, "action": "🥞 Eat pancakes with syrup", "impact": 30, "glucose_level": 130},
        {"user_id": 1, "action": "🚶‍♂️ Take a walk", "impact": -15, "glucose_level": 115},
        {"user_id": 2, "action": "🍗 Have grilled chicken and veggies", "impact": 5, "glucose_level": 105},
    ]
    for data in sample_logs:
        log = GlucoseLog(
            user_id=data["user_id"],
            action=data["action"],
            impact=data["impact"],
            glucose_level=data["glucose_level"]
        )
        db.session.add(log)
    db.session.commit()
```
---

## User Stories

### 1. Patient Perspective — *"Check My Glucose"*

**As a** health-conscious player,  
**I want to** see how my daily choices impact glucose  
**So that** I can make better decisions in real life

**Acceptance Criteria**:
- Submit lifestyle choices  
- See glucose updates after each choice  
- Receive Dexcom-style alerts and feedback  
- Earn badges for good decision streaks

---

### 2. Doctor Perspective — *"Rapid Screening"*

**As a** healthcare provider,  
**I want to** view a patient’s choice log  
**So that** I can identify risky behavior patterns

**Acceptance Criteria**:
- View user logs via API  
- See glucose trend history  
- Flag dangerous spikes (>200 mg/dL)

---

### 3. Clinic Admin — *"Population Insights"*

**As a** clinic manager,  
**I want to** analyze log data across users  
**So that** I can optimize public health campaigns

**Acceptance Criteria**:
- Anonymized logs via backend  
- Export reports  
- Demographic risk breakdowns

---

### 4. Public Health Researcher — *"Model Transparency"*

**As a** researcher,  
**I want to** analyze how behavior impacts glucose  
**So that** I can validate educational effectiveness

**Acceptance Criteria**:
- Export logs + actions + outcomes  
- Evaluate engagement per scenario  
- Compare to known diabetes triggers

---

### 5. Dexcom Marketing — *"Gamified Awareness"*

**As a** social media lead,  
**I want to** turn CGM insights into a game  
**So that** users engage and share their experience

**Acceptance Criteria**:
- Dexcom-style feedback animations  
- Social sharing of streaks / scores  
- Badge system for perfect choices  

---

## Next Steps

- [ ] Finalize event bank (choices + glucose impact)  
- [ ] Finish component structure in React  
- [ ] Implement glucose logic + animations  
- [ ] Run internal beta tests  
- [ ] Launch + track engagement metrics  

---

## Goals

> Launch goal: Increase Dexcom-related social media engagement by **30% in 3 months**  
> Long-term: Improve diabetes education through relatable gamification
